# Test spike train distance EFeature

Compare spike trains obtained from protocol responses using spike train distance metrics.

In [1]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook

# print code version (hash of checked out version)
!git log -1

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..
%pwd

commit 8cb10a93f5ef55fec995fba3c5b9371e093b223a (HEAD -> nothreadsafe, origin/nothreadsafe)
Author: Lucas Koelman <lucas.koelman@gmail.com>
Date:   Sat Oct 7 16:23:43 2017 +0100

    Add ephys protocol extension class to plug in shared protocol functions

Notebook executed at at 2017-10-09 13:19:13.083640 in following directory:
/home/luye/workspace/bgcellmodels/GilliesWillshaw


u'/home/luye/workspace/bgcellmodels/GilliesWillshaw'

In [2]:
# BluePyOpt
import bluepyopt.ephys as ephys

# Our custom BluePyOpt modules
from optimize.bpop_cellmodels import StnFullModel, StnReducedModel

import optimize.bpop_protocols_stn as StnProtocols
from optimize.bpop_protocols_stn import BpopProtocolWrapper

import optimize.bpop_features_stn as StnFeatures
import optimize.bpop_parameters_stn as StnParameters

from optimize.bpop_analysis_stn import (
    run_proto_responses, plot_proto_responses, 
    save_proto_responses, load_proto_responses,
    plot_responses
)

# Physiology parameters
from evalmodel.cellpopdata import StnModel
from evalmodel.proto_common import StimProtocol

CLAMP_PLATEAU = StimProtocol.CLAMP_PLATEAU
CLAMP_REBOUND = StimProtocol.CLAMP_REBOUND
MIN_SYN_BURST = StimProtocol.MIN_SYN_BURST

In [3]:
# Adjust verbosity of loggers
import logging
silent_loggers = ['marasco', 'redops', 'folding', 'bluepyopt.ephys.parameters']
verbose_loggers = []
for logname in silent_loggers:
	logger = logging.getLogger(logname)
	if logger: logger.setLevel(logging.WARNING)

## Make protocols and parameters

In [4]:
frozen_params = [] # SETPARAM: frozen params from previous optimisations
free_params = [] # StnParameters.dend_active_params # SETPARAM: parameters that are optimised (must be not frozen)


# Protocols for optimisation
stn_model_type = StnModel.Gillies_FoldMarasco # SETPARAM: model type to optimise

# Protocols to use for optimisation
opt_stim_protocols = [CLAMP_REBOUND, MIN_SYN_BURST]
red_protos = {stim_proto: BpopProtocolWrapper.make(stim_proto, stn_model_type) for stim_proto in opt_stim_protocols}

# Collect al frozen mechanisms and parameters required for protocols to work
proto_mechs, proto_params = BpopProtocolWrapper.all_mechs_params(red_protos.values())

# Distinguish between sets of parameters (used, frozen, free/optimised)
frozen_params += proto_params
used_params = frozen_params + free_params

for param in frozen_params: assert param.frozen
for param in free_params: assert (not param.frozen)

## Features for full model

In [5]:
# Get protocol responses for full model
PROTO_RESPONSES_FILE = "/home/luye/cloudstore_m/simdata/fullmodel/STN_Gillies2005_proto_responses.pkl" # backup is in filename.old.pkl
full_responses = load_proto_responses(PROTO_RESPONSES_FILE)

# Make EFEL feature objects
stimprotos_feats = StnFeatures.make_opt_features(red_protos.values()) # dict {stim_protocol : (efeature, weight)}

# Calculate target values from full model responses
StnFeatures.calc_feature_targets(stimprotos_feats, full_responses)

bluepyopt.ephys.efeatures:DEBUG:Calculated value for CLAMP_REBOUND.min_AHP_values: -61.2351049487 @efeatures.py:177
bpop_ext:DEBUG:Calculated value for CLAMP_REBOUND.Victor_Purpura_distance: [ 511.   517.5  522.2  528.   533.8  539.9  546.4  552.9  559.5  566.1
  572.8  579.6  586.4  593.3  600.3  607.7  617. ] @bpop_efeatures.py:238
bluepyopt.ephys.efeatures:DEBUG:Calculated value for CLAMP_REBOUND.ISI_CV: 0.14756131579 @efeatures.py:177
bluepyopt.ephys.efeatures:DEBUG:Calculated value for CLAMP_REBOUND.adaptation_index: 0.0119738183039 @efeatures.py:177
bluepyopt.ephys.efeatures:DEBUG:Calculated value for CLAMP_REBOUND.Spikecount: 17.0 @efeatures.py:177
bluepyopt.ephys.efeatures:DEBUG:Calculated value for MIN_SYN_BURST.min_AHP_values: -64.4950936613 @efeatures.py:177
bpop_ext:DEBUG:Calculated value for MIN_SYN_BURST.Victor_Purpura_distance: [   3.8   10.    15.5   21.6   27.6   33.6   39.8   45.9   52.1   58.3
   64.6   70.8   77.1   83.4   89.8   96.2  102.5  108.9  115.4  121.8
  1

## Features for reduced model

In [6]:
# Create reduced model and get parameters
red_model = StnReducedModel(
                name		= 'StnFolded',
                fold_method	= 'marasco',
                num_passes	= 7,
                mechs		= proto_mechs,
                params		= used_params)

default_params = {k : v*1.1 for k,v in StnParameters.default_params.items() if k in [p.name for p in used_params]}

nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

# Choose protocol
stim_proto = MIN_SYN_BURST
e_proto = red_protos[stim_proto]

for stim_proto in opt_stim_protocols:

    # Compute protocol responses
    responses = e_proto.ephys_protocol.run(
                    cell_model		= red_model, 
                    param_values	= {},
                    sim				= nrnsim,
                    isolate			= False) # allows us to query cell with h.allsec()

    # Plot responses
    print("Full model responses:")
    plot_responses(full_responses[stim_proto.name])

    print("Reduced model responses:")
    plot_responses(responses)

    # Calculate response scores
    featdict = stimprotos_feats[stim_proto]
    
    feat_names, feat_data = zip(*featdict.items())
    efeats, weights = zip(*feat_data)

    for efeat in efeats:
        score = efeat.calculate_score(responses)
        print('Score = <{}> for feature {}.{}'.format(score, stim_proto.name, efeat.name))

bluepyopt.ephys.mechanisms:DEBUG:Inserted GLUsyn at ['zipF_zipE_zipD_SThcell0dend01(0.9375)', 'zipG_zipF_zipE_zipD_SThcell0dend02(1)', 'zipG_zipF_zipE_zipD_SThcell0dend02(1)', 'zipG_zipF_zipE_zipD_SThcell0dend01(1)']  @mechanisms.py:303
bluepyopt.ephys.mechanisms:DEBUG:Inserted GABAsyn at ['SThcell[0].dend0[2](0.75)']  @mechanisms.py:303
bpop_ext:DEBUG:Set aCSF concentrations. @bpop_protocols_stn.py:388
bluepyopt.ephys.recordings:DEBUG:Adding compartment recording of v at somatic[0](0.5) @recordings.py:89
bluepyopt.ephys.simulators:DEBUG:Running Neuron simulator 1050 ms, with dt=0.025 @simulators.py:133


1  
loading STh cell data structures...
 
1 Creating STh Neurons

22   2 trees
  loaded tree 0 (23 branches)
  loaded tree 1 (11 branches)
loading linear+uniform conductances
0.0001 0.001 Skipping soma


bluepyopt.ephys.simulators:DEBUG:Neuron simulation finished @simulators.py:147


Full model responses:


<IPython.core.display.Javascript object>

Reduced model responses:


<IPython.core.display.Javascript object>

bluepyopt.ephys.efeatures:DEBUG:Recording named CLAMP_REBOUND.soma.v not found in responses {'MIN_SYN_BURST.soma.v': <bluepyopt.ephys.responses.TimeVoltageResponse object at 0x7f2208f26050>} @efeatures.py:118
bluepyopt.ephys.efeatures:DEBUG:Calculated score for CLAMP_REBOUND.min_AHP_values: 250.000000 @efeatures.py:205
bpop_ext:DEBUG:Recording named CLAMP_REBOUND.soma.v not found in responses {'MIN_SYN_BURST.soma.v': <bluepyopt.ephys.responses.TimeVoltageResponse object at 0x7f2208f26050>} @bpop_efeatures.py:180
bpop_ext:DEBUG:Calculated score for CLAMP_REBOUND.Victor_Purpura_distance: 250.000000 @bpop_efeatures.py:286
bluepyopt.ephys.efeatures:DEBUG:Recording named CLAMP_REBOUND.soma.v not found in responses {'MIN_SYN_BURST.soma.v': <bluepyopt.ephys.responses.TimeVoltageResponse object at 0x7f2208f26050>} @efeatures.py:118
bluepyopt.ephys.efeatures:DEBUG:Calculated score for CLAMP_REBOUND.ISI_CV: 250.000000 @efeatures.py:205
bluepyopt.ephys.efeatures:DEBUG:Recording named CLAMP_REBOUN

Score = <250> for feature CLAMP_REBOUND.CLAMP_REBOUND.min_AHP_values
Score = <250> for feature CLAMP_REBOUND.CLAMP_REBOUND.Victor_Purpura_distance
Score = <250> for feature CLAMP_REBOUND.CLAMP_REBOUND.ISI_CV
Score = <250> for feature CLAMP_REBOUND.CLAMP_REBOUND.adaptation_index
Score = <250> for feature CLAMP_REBOUND.CLAMP_REBOUND.Spikecount


Exception: Traceback (most recent call last):
  File "/home/luye/workspace/BluePyOpt/bluepyopt/ephys/protocols.py", line 132, in _run_func
    self.instantiate(sim=sim, icell=cell_model.icell)
  File "optimize/bpop_extensions.py", line 287, in instantiate
    icell=icell)
  File "/home/luye/workspace/BluePyOpt/bluepyopt/ephys/protocols.py", line 202, in instantiate
    stimulus.instantiate(sim=sim, icell=icell)
  File "/home/luye/workspace/BluePyOpt/bluepyopt/ephys/stimuli.py", line 134, in instantiate
    self.connections[location.name] = []
TypeError: 'NoneType' object does not support item assignment
